# 1. Set up the notebook

Import modules

In [ ]:
import numpy as np
import sympy as sym
import json
import matplotlib.pyplot as plt
from scipy import linalg
from scipy.interpolate import interp1d

Define a function to load data from hardware (and resample it at 100 Hz using linear interpolation).

In [ ]:
def load_hardware_data(filename, t_min_offset=0, t_max_offset=0):
    # load raw data
    with open(filename, 'r') as f:
        data = json.load(f)

    # convert lists to numpy arrays
    for val in data.values():
        for key in val.keys():
            val[key] = np.array(val[key])

    # create an array of times at which to subsample
    t_min = -np.inf
    t_max = np.inf
    for key, val in data.items():
        t_min = max(t_min, val['time'][0])
        t_max = min(t_max, val['time'][-1])
    t_min += t_min_offset * 1000
    t_max -= t_max_offset * 1000
    nt = int(1 + np.floor((t_max - t_min) / 10.))
    t = np.arange(0, 10 * nt, 10) / 1000.
    resampled_data = {'time': t}

    # resample raw data with linear interpolation
    for k, v in data.items():
        f = interp1d((v['time'] - t_min) / 1000., v['data'])
        resampled_data[k] = f(t)
        
    # return the resampled data
    return resampled_data

# 2. Study accelerometer measurements

## 2.1 Drone sitting on the table

Do the following:

* Put the drone on a table.
* Run the client software (no move commands).
* Change name of data file to `on_the_table.json`.

Then, load and parse the data.

In [ ]:
data = load_hardware_data('on_the_table.json')

# time
t = data['time']

# accelerometer measurements
a_x = data['acc.x']
a_y = data['acc.y']
a_z = data['acc.z']

Plot raw accelerometer measurements as a function of time.

In [ ]:
plt.figure(figsize=(10, 8))
plt.plot(t, a_x, '--', label='a_x', linewidth=2)
plt.plot(t, a_y, '-.', label='a_y', linewidth=2)
plt.plot(t, a_z, '-', label='a_z', linewidth=2)
plt.legend(fontsize=16)
plt.grid()
plt.tick_params(axis='x', labelsize=16)
plt.tick_params(axis='y', labelsize=16)
plt.ylabel('specific force (units?)', fontsize=16)
plt.xlabel('time (s)', fontsize=16)
plt.ylim([-2, 2])

Do the following:

* Correctly label the units of specific force in the plot above.
* Scale the accelerometer measurements and plot them again below (with correctly labeled units).

In [ ]:
# accelerometer measurements
a_x = data['acc.x'] # <-- FIXME
a_y = data['acc.y'] # <-- FIXME
a_z = data['acc.z'] # <-- FIXME

plt.figure(figsize=(10, 8))
plt.plot(t, a_x, '--', label='a_x', linewidth=2)
plt.plot(t, a_y, '-.', label='a_y', linewidth=2)
plt.plot(t, a_z, '-', label='a_z', linewidth=2)
plt.legend(fontsize=16)
plt.grid()
plt.tick_params(axis='x', labelsize=16)
plt.tick_params(axis='y', labelsize=16)
plt.ylabel('specific force (units?)', fontsize=16)
plt.xlabel('time (s)', fontsize=16)
plt.ylim([-20, 20])

# 2.2 Drone in free fall

Do the following:

* Put the drone on the table.
* Run the client software (no move commands).
* Pick up the drone.
* Drop the drone.
* Change name of data file to `in_free_fall.json`.

Then, load and parse (and scale) the data.

In [ ]:
data = load_hardware_data('in_free_fall.json')

# time
t = data['time']

# accelerometer measurements
g = 9.81
a_x = g * data['acc.x']
a_y = g * data['acc.y']
a_z = g * data['acc.z']

Plot scaled accelerometer measurements as a function of time, and identify the time intervals in which the drone was...

* Sitting on the table (as originally placed by hand)
* Being picked up and moved around by hand
* In free fall
* Sitting on the table (after falling)

In [ ]:
plt.figure(figsize=(10, 8))
plt.plot(t, a_x, '--', label='a_x', linewidth=2)
plt.plot(t, a_y, '-.', label='a_y', linewidth=2)
plt.plot(t, a_z, '-', label='a_z', linewidth=2)
plt.legend(fontsize=16)
plt.grid()
plt.tick_params(axis='x', labelsize=16)
plt.tick_params(axis='y', labelsize=16)
plt.ylabel('specific force (units?)', fontsize=16)
plt.xlabel('time (s)', fontsize=16)
plt.ylim([-20, 20])
# plt.xlim([2, 3])

# 2.3 Drone sitting at an angle

Do the following:

* Put the drone on a flat, angled surface (e.g., an ipad case).
* Run the client software (no move commands).
* Change name of data file to `at_an_angle.json`.

Then, based on a guess at the drone's orientation, predict the accelerometer measurements.

Finally, load and parse (and scale) the data.

In [ ]:
data = load_hardware_data('at_an_angle.json')

# time
t = data['time']

# accelerometer measurements
g = 9.81
a_x = g * data['acc.x']
a_y = g * data['acc.y']
a_z = g * data['acc.z']

Plot scaled accelerometer measurements as a function of time, and compare predictions to actual measurements.

In [ ]:
plt.figure(figsize=(10, 8))
plt.plot(t, a_x, '--', label='a_x', linewidth=2)
plt.plot(t, a_y, '-.', label='a_y', linewidth=2)
plt.plot(t, a_z, '-', label='a_z', linewidth=2)
plt.legend(fontsize=16)
plt.grid()
plt.tick_params(axis='x', labelsize=16)
plt.tick_params(axis='y', labelsize=16)
plt.ylabel('specific force (units?)', fontsize=16)
plt.xlabel('time (s)', fontsize=16)
plt.ylim([-20, 20])

Compute the angle of the drone.

For example, suppose it was rotated about the pitch axis. Then:

$$\begin{align*} a_x &= -g \sin(\theta) \\ a_z &= g \cos(\theta) \end{align*}$$

We can solve for $\sin\theta$ and $\cos\theta$:

$$\begin{align*} \sin(\theta) &= - a_x / g \\ \cos(\theta) &= a_z / g \end{align*}$$

Then, we can solve for $\theta$ using the four-quadrant arc-tangent function:

In [ ]:
theta = np.arctan2(-a_x / g, a_z / g)

Plot $\theta$ as a function of time in degrees.

In [ ]:
plt.figure(figsize=(10, 8))
plt.plot(t, np.rad2deg(theta), label='theta', linewidth=2)
plt.legend(fontsize=16)
plt.grid()
plt.tick_params(axis='x', labelsize=16)
plt.tick_params(axis='y', labelsize=16)
plt.ylabel('angle (degrees)', fontsize=16)
plt.xlabel('time (s)', fontsize=16)
plt.ylim([-90, 90])

Show the mean value of $\theta$ (the mean angle) in degrees:

In [ ]:
print(f'theta = {np.rad2deg(np.mean(theta)):.1f} degrees')

# 2.4 Drone moving while at an angle

Do the following:

* Put the drone on a flat, angled surface (e.g., an ipad case).
* Run the client software (no move commands).
* Move the surface.
* Change name of data file to `moving_at_an_angle.json`.

Then guess how the accelerometer measurements will be different from what you saw in Section 2.3.

Finally, load and parse (and scale) the data.

In [ ]:
data = load_hardware_data('moving_at_an_angle.json')

# time
t = data['time']

# accelerometer measurements
g = 9.81
a_x = g * data['acc.x']
a_y = g * data['acc.y']
a_z = g * data['acc.z']

Plot scaled accelerometer measurements as a function of time. Do they make sense? Associate time intervals to parts of the experiment.

In [ ]:
plt.figure(figsize=(10, 8))
plt.plot(t, a_x, '--', label='a_x', linewidth=2)
plt.plot(t, a_y, '-.', label='a_y', linewidth=2)
plt.plot(t, a_z, '-', label='a_z', linewidth=2)
plt.legend(fontsize=16)
plt.grid()
plt.tick_params(axis='x', labelsize=16)
plt.tick_params(axis='y', labelsize=16)
plt.ylabel('specific force (units?)', fontsize=16)
plt.xlabel('time (s)', fontsize=16)
plt.ylim([-20, 20])

Attempt to compute the angle of the drone, just like before.

In [ ]:
theta = np.arctan2(-a_x / g, a_z / g)

Plot $\theta$ as a function of time in degrees.

In [ ]:
plt.figure(figsize=(10, 8))
plt.plot(t, np.rad2deg(theta), label='theta', linewidth=2)
plt.legend(fontsize=16)
plt.grid()
plt.tick_params(axis='x', labelsize=16)
plt.tick_params(axis='y', labelsize=16)
plt.ylabel('angle (degrees)', fontsize=16)
plt.xlabel('time (s)', fontsize=16)
plt.ylim([-90, 90])

So, are we able to use accelerometer measurements to estimate the pitch and roll angles directly? (In what applications *could* we do this?)